### Reference Notbooks
4. Natural language Processing IV (Section 5.4 thne 5.6)
4. NLP Exercise - do with student

### Steps to Follow
1. Try NaiveBayes classiffier first
2. Data Cleaning, lemming, etc
3. TFIDF
5. Try Random Forest (or K-means with only 2 clases?)

In [1]:
import pandas as pd
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
# Importing the Training Data file
df= pd.read_csv('TRAINING_DATA.txt', sep='\t')
df.columns = ['tag', 'text']
df.sample(10)

,tag,text
9835,0,Nota del editor : Tomio Geron es la cabeza de ...
1444,1,Él fue el único de los últimos nueve jugadores...
12243,0,David y yo no podría haber sido más contentos...
1794,1,Escribir usando cualquier tipo de mando a dist...
9526,0,"Lo que significa , querido tío recta imaginari..."
3095,1,La final de El Soltero era una que los fanes n...
6321,0,Pronto me enteré de lo que mi padre ya sabía :...
14589,1,Mi novio y yo hemos estado juntos desde de la ...
13496,0,Más grande de China compañía de comercio elect...
4824,0,Era fácil para deshacerse de muchas de estas p...


In [3]:
def clean_data(text):        
    text = ''.join(f' {e} ' if e in string.punctuation else e for e in text)  # Add spaces around special characters
    text = ''.join(e for e in text if e.isalpha() or e.isspace())  # Remove special characters and numbers
    text = ' '.join(word for word in text.split() if len(word) > 1)  # Remove single characters
    text = text.lstrip()  # Remove single characters from the start
    text = ' '.join(text.split())  # Substitute multiple spaces with single space
    text = text.replace('b ', '')  # Remove prefixed 'b'
    text = text.lower()  # Convert to lowercase
    text = text.strip()  # Remove leading/trailing whitespace
    
    return text

clean_df = df
clean_df['text'] = df['text'].apply(clean_data)
clean_df.sample(10)

,tag,text
3612,1,ambos productos serán lanzados en todo el mund...
3258,1,al igual que las dos primeras partes del estud...
3557,0,después de los clientes mantienen preguntando ...
2950,0,en marzo después de que amber había pasado un ...
5756,1,en ningún momento durante el proceso sentí que...
4278,0,la mayoría de la stasi no torturadores bestias...
12991,0,real housewives of gretchen rossi del condado ...
7998,1,este es el mismo hockey cuyo propio presupuest...
2160,0,te gustaría que la gente que camina usted apun...
10015,1,el estuche flir one le da su iphone una visión...


In [4]:
# Get the Spanish stopwords
stop_words = set(stopwords.words('spanish'))
# Remove stopwords
no_sw_df = clean_df
clean_df['text'] = clean_df['text'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

In [ ]:
### TODO: Lemmatization

In [ ]:
### TODO: Bag of Words?

In [13]:
#Defining target and features
X = df['text']
y = df['tag']

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
### TODO: Bag of Words or some way of vectorizing the data
countvector=CountVectorizer(ngram_range=(2,2))
traindataset=countvector.fit_transform(headlines)

In [ ]:
### Testing Naive Bayes out of the box
#Creating and Training the Naive bayes model
nb_model = MultinomialNB().fit(X_train, y_train)

In [ ]:
### Making predictions and Evaluating the model
## Predict for the Test Dataset
y_pred = nb_model.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(matrix)

acc_score = accuracy_score(X_test, y_pred)
print(f"Accuracy Score: {acc_score:.4f}")

report = classification_report(X_test, y_pred)
print(report)

In [ ]:
""" # Creating a confusion matrix
cm = confusion_matrix(
    np.argmax(y_test, axis=1),                                                 # True class labels converted from one-hot encoding to class indices
    np.argmax(y_pred, axis=1)                                                  # Predicted class labels converted from one-hot encoding to class indices
)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))                                                     #   Set figure size to make the plot readable
plt.imshow(cm, interpolation='nearest', cmap='Greens')                         # Display the confusion matrix with a green color map
plt.title('Confusion Matrix')                                                  # Set the title of the plot
plt.colorbar()                                                                 # Add a color bar to interpret the matrix values visually

# Use the custom class names as tick labels
plt.xticks(range(len(custom_class_names)), custom_class_names, rotation=45)    # Set x-axis tick marks and labels with a 45-degree rotation
plt.yticks(range(len(custom_class_names)), custom_class_names)                 # Set y-axis tick marks and labels

# Add numbers to the confusion matrix
for i in range(cm.shape[0]):                                                   # Iterate over rows
    for j in range(cm.shape[1]):                                               # Iterate over columns
        plt.text(j, i, cm[i, j], ha='center', va='center', color='black')      # Add the count in each cell with black text

plt.show()  # Display the confusion matrix plot """